# Metadata Vignette

## Counting HCA attribute usage

### By Matthew Green


The HCA has tightly controlled, well defined, curated metadata assiciated with all it's projects. There are nearly 600 fields defined by our JSON metadata. But which ones are most frequently used? This vignette answers this question by doing the following:

This code does the following:

1. Fetches latest attributes from metadata schema
1. Defines an example of an ES query for specific bundles (in the provided example we return all projects that are not 'black listed') and makes a generator from the DSS for bundle iteration
1. Counts these attributes in the bundles from the geenrator
1. Makes a few graphs and tables (there are options to save these.

## 1.

This section queries the metadata API to get the fields from the latest version of the schema. The list returned are the programatic names of the attributes. 

**The Query against the metadata API may take a couple of minutes.**

In [1]:
from ingest.template.schema_template import SchemaTemplate
import yaml

template = SchemaTemplate()
attribute_yaml = template.yaml_dump()

attribute_list = yaml.load(attribute_yaml).get('tabs')
full_set = set()
full_list = []
for tab in attribute_list:
    for x in list(tab.values()):
        for attribute in x.get('columns'):
            full_set.add(attribute)
            full_list.append(attribute)
print('{} out of {} attributes are unique'.format(len(full_set), len(full_list)))
print(full_list)

590 out of 590 attributes are unique
['sequencing_protocol.protocol_core.protocol_id', 'sequencing_protocol.protocol_core.protocol_name', 'sequencing_protocol.protocol_core.protocol_description', 'sequencing_protocol.protocol_core.publication_doi', 'sequencing_protocol.protocol_core.protocols_io_doi', 'sequencing_protocol.protocol_core.document', 'sequencing_protocol.instrument_manufacturer_model.text', 'sequencing_protocol.instrument_manufacturer_model.ontology', 'sequencing_protocol.instrument_manufacturer_model.ontology_label', 'sequencing_protocol.local_machine_name', 'sequencing_protocol.paired_end', 'sequencing_protocol.method.text', 'sequencing_protocol.method.ontology', 'sequencing_protocol.method.ontology_label', 'sequencing_protocol.10x.fastq_method', 'sequencing_protocol.10x.fastq_method_version', 'sequencing_protocol.10x.pooled_channels', 'sequencing_protocol.10x.drop_uniformity', 'library_preparation_protocol.protocol_core.protocol_id', 'library_preparation_protocol.protoc

In [2]:
# this is a temp cell to add some out of date attributes that are present in the cbeta

full_list = full_list + ['dissociation_protocol.method.ontology_label',
'library_preparation_protocol.library_construction_method.text',
'sequencing_protocol.method.text',
'dissociation_protocol.method.text',
'sequencing_protocol.method.ontology',
'specimen_from_organism.organ_parts.text',
'specimen_from_organism.organ_parts.ontology_label',
'library_preparation_protocol.library_construction_method.ontology_label',
'specimen_from_organism.organ_parts.ontology',
'sequencing_protocol.method.ontology_label',
'library_preparation_protocol.library_construction_method.ontology',
'sequence_file.insdc_run_accessions',
'cell_suspension.estimated_cell_count',
'donor_organism.human_specific.ethnicity.text',
'project.insdc_project_accessions',
'project.geo_series_accessions',
'project.funders.grant_id',
'process.insdc_experiment.insdc_experiment_accession',
'cell_suspension.plate_based_sequencing.well_quality',
'collection_protocol.method.ontology',
'project.insdc_study_accessions',
'collection_protocol.method.text',
'collection_protocol.method.ontology_label',
'cell_suspension.plate_based_sequencing.plate_label',
'project.array_express_accessions',
'cell_suspension.plate_based_sequencing.well_quality',
'donor_organism.familial_relationships.child',
'donor_organism.familial_relationships.parent',
'cell_line.cell_morphology.cell_size_unit.text',
'cell_suspension.cell_morphology.cell_size_unit.text',
'ipsc_induction_protocol.protocol_reagents.catalog_number',
'ipsc_induction_protocol.protocol_reagents.manufacturer',
'ipsc_induction_protocol.protocol_reagents.expiry_date',
'ipsc_induction_protocol.protocol_reagents.lot_number',
'ipsc_induction_protocol.protocol_reagents.retail_name',
'donor_organism.mouse_specific.strain.text',
'dissociation_protocol.method.ontology']

## 2a.

`initial_q` is the initial Elastic Search query we will use to get the bundles we will feed to the attribute counter in the next step. 

The top example contains a `must_not` term that will exclude datasets taken from the exclusion list [here](https://github.com/DataBiosphere/azul/blob/c0937a7dcbd6f70f6d28ce004f4005c8eac0d9dc/src/azul/project/hca/__init__.py#L90).

**(Reccomended) The bottom example contains a `must` term that will only include projects. These datasets were taben from [here](https://docs.google.com/spreadsheets/d/155ACGfrgobXy9DI_GduYyHCpkCyFDIa6FvgH8keBdug/edit?ts=5c084ebb#gid=287685589)**

You can also add `must` so this query only matches specific attributes or specific values. For example you may only want to match human smart seq 2 datasets. For this you could use queries similar to [this one](https://github.com/HumanCellAtlas/lira/blob/master/scripts/vx_queries/smartseq2-query.json) used by our internal analysis pipeline.

In [3]:
initial_q = {
    "query": {
        "bool": {
            "must_not": [
                {
                    "terms": {
                        "files.project_json.provenance.document_id": [
                            "1630e3dc-5501-4faf-9726-2e2c0b4da6d7",
                            "fd1d163d-d6a7-41cd-b3bc-9d77ba9a36fe",
                            "2a0faf83-e342-4b1c-bb9b-cf1d1147f3bb",
                            "cf8439db-fcc9-44a8-b66f-8ffbf729bffa",
                            "6b9f514d-d738-403f-a9c2-62580bbe5c83",
                            "311d013c-01e4-42c0-9c2d-25472afa9cbc",
                            "d237ed6a-3a7f-4a91-b300-b070888a8542",
                            "e6cc0b02-2125-4faa-9903-a9025a62efec",
                            "e4dbcb98-0562-4071-8bea-5e8de5f3c147",
                            "e79e9284-c337-4dfd-853d-66fa3facfbbd",
                            "560cd061-9165-4699-bc6e-8253e164c079",
                            "e83fda0e-6515-4f13-82cb-a5860ecfc2d4",
                            "9a60e8c2-32ea-4586-bc1f-7ee58f462b07",
                            "71a6e049-4846-4c2a-8823-cc193c573efc",
                            "4b5a2268-507c-46e6-bab0-3efb30145e85",
                            "364ebb73-652e-4d32-8938-1c922d0b2584",
                            "11f5d59b-0e2c-4f01-85ac-8d8dd3db53be",
                            "c1996526-6466-40ff-820f-dad4d63492ec",
                            "c281dedc-e838-4464-bf51-1cc4efae3fb9",
                            "40afcf6b-422a-47ba-ba7a-33678c949b5c",
                            "71a6e049-4846-4c2a-8823-cc193c573efc",
                            "9a60e8c2-32ea-4586-bc1f-7ee58f462b07",
                            "0facfacd-5b0c-4228-8be5-37aa1f3a269d",
                            "76c209df-42bf-41dc-a5f5-3d27193ca7a6",
                            "bb409c34-bb87-4ed2-adaf-6d1ef10610b5",
                            "1a6b5e5d-914f-4dd6-8817-a1f9b7f364d5",
                            "dd401943-1059-4b2d-b187-7a9e11822f95"
                        ]
                    }
                }
            ]
        }
    }
}

In [4]:
initial_q = {
    "query": {
        "bool": {
            "must": [
                {
                    "terms": {
                        "files.project_json.provenance.document_id": [
                            "d96c2451-6e22-441f-a3e6-70fd0878bb1b",
                            "e8642221-4c2c-4fd7-b926-a68bce363c88",
                            "209e6402-d854-49ea-815f-421dae5e3f4d",
                            "29f53b7e-071b-44b5-998a-0ae70d0229a4",
                            "f8880be0-210c-4aa3-9348-f5a423e07421",
                            "aabbec1a-1215-43e1-8e42-6489af25c12c",
                            "179bf9e6-5b33-4c5b-ae26-96c7270976b8",
                            "1a0f98b8-746a-489d-8af9-d5c657482aab",
                            "f396fa53-2a2d-4b8a-ad18-03bf4bd46833",
                            "0ec2b05f-ddbe-4e5a-b30f-e81f4b1e330c",
                            "32eb86db-6842-480f-a49a-a2b0161ed35a",
                            "0c7bbbce-3c70-4d6b-a443-1b92c1f205c8",
                            "34ec62a2-9643-430d-b41a-1e342bd615fc",
                            "c765e3f9-7cfc-4501-8832-79e5f7abd321",
                            "ff481f29-3d0b-4533-9de2-a760c61c162d",
                            "5f256182-5dfc-4070-8404-f6fa71d37c73"
                        ]
                    }
                }
            ]
        }
    }
}

## 2b.

Making a bundle generator from the DSS. This geenrator will iterate through all the matches but it has optimisation behind the scenes to balance payload per request and number of requests for you.

You'll need to `pip install hca` on your machine to use these modules.

In [5]:
import hca.dss
from hca.dss import DSSClient

dss_client = DSSClient(swagger_url= "https://dss.data.humancellatlas.org/v1/swagger.json")
bundle_generator = dss_client.post_search.iterate(replica="aws", es_query=initial_q, output_format="raw")
total_hits = dss_client.post_search(replica="aws", es_query=initial_q, output_format="raw").get('total_hits')

## 3.

In section 3 we will iterate through bundles captured in section 2. For each bundle we will count attributes from the list generated in section 1 then convert these counts into a per project count rather than just per bundle.

**As the iterator contains all the bundles in the data store, this counting can take 1-2h** If you want to quickly test the script adjust the `LIMIT_RUN` number to limit the number of bundles you want to process. If you set this to `False` all the bundles that match the query will be processed.

In [6]:
LIMIT_RUN = False
# LIMIT_RUN = 1000


import sys # temp remove now
import json # temp remove now
import pandas as pd
from tqdm import tqdm_notebook as tqdm

temp_counter = 0 # remove for full run
attribute_present_all = {}
project_mapping = {}
for bundle in tqdm(bundle_generator, total=total_hits, unit='bundle'):
    
    bundle_url = bundle.get('bundle_url')
    bundle_meta = bundle.get('metadata').get('files')
    attribute_present = {}
    missing_bundle_counter = 0
    if bundle_meta:

        # make project uuid to bundle uuid mapping dict
        try:
            project_uuid = bundle_meta.get('project_json')[0].get('provenance').get('document_id')
        except KeyError:
            project_uuid = bundle_meta.get('project_json').get('hca_ingest').get('document_id')
            
        if project_uuid in project_mapping:
            old_value = project_mapping.get(project_uuid)
            new_value = old_value + [bundle_url]
            project_mapping[project_uuid] = new_value
            
        else:
            project_mapping[project_uuid] = [bundle_url]
   
        # search for all attributes in the bundle
        
        for attribute in full_list:
            nesting = attribute.split('.')
            nesting[0] = nesting[0] + '_json'
            nested_get = bundle_meta.get(nesting[0])
            
            if nested_get:
                for metadata_entity in nested_get:
                    for idx, level in enumerate(nesting[1:]):
                    
                        try:
                            if attribute_present.get(attribute) == True: # if attribute already found in bundle
                                if idx == len(nesting[1:]):
                                    break
                            metadata_entity = metadata_entity.get(level)
                            attribute_present[attribute] = True
                            
                            
                        except AttributeError:
                            attribute_present[attribute] = False
  
            else:
                attribute_present[attribute] = False
    else:
        missing_bundle_counter += 1
    attribute_present_all[bundle_url] = (attribute_present)
    temp_counter += 1 # remove for full run
    if temp_counter == LIMIT_RUN:
        break
        
    
print('MISSING {} BUNDLES'.format(missing_bundle_counter))

df = pd.DataFrame(attribute_present_all)
df.to_csv('temp_df.csv') # save out for inspection
print('DataFrame contains {} rows/attributes and {} columns/bundles'.format(df.shape[0], df.shape[1]))


MISSING 0 BUNDLES
DataFrame contains 590 rows/attributes and 42254 columns/bundles


## 3b.

This section counts up the results from the dataframe created above.

In [7]:
bundle_counts = df.count(axis=1, level=None, numeric_only=False)
# print(bundle_counts)
# bundle_counts.to_csv('bundle_counts.csv')


bundle_counts = {}

for attribute in tqdm(full_list, unit='attribute'):
    value_counts = df.T[attribute].value_counts()
    true_counts = value_counts.get(True)
    false_counts = value_counts.get(False)
    
    bundle_counts[attribute] = {'True': true_counts, 'False' : false_counts}
    
# print(bundle_counts) # a dictionary containing counts

## 3c.

In the previous section we attribute usage in bundles but ideally we want to know the counts on a project basis. Using the bundle to project mapping `project_mapping` we created earler, here we go...

In [8]:
project_bool = {}

for project_uuid, bundles in project_mapping.items():
    project_df = df[bundles]
    for attribute in full_list:
        attribute_exists = project_df.T[attribute].any()
        
        if project_uuid in project_bool:
            old_value = project_bool.get(project_uuid)
            new_value = old_value[attribute] = attribute_exists
        else:
            project_bool[project_uuid] = {attribute : attribute_exists}
        
project_df = pd.DataFrame(project_bool)
        
project_counts = {}

for attribute in tqdm(full_list, unit='attribute'):
    value_counts = project_df.T[attribute].value_counts()
    true_counts = value_counts.get(True)
    false_counts = value_counts.get(False)
    
    project_counts[attribute] = {'True': true_counts, 'False' : false_counts} 
        
# print(project_counts) # a dictionary containing counts
project_counts_df = pd.DataFrame(project_counts).T.fillna(value=0)
# project_counts_df.to_csv('project_counts.csv')

# add columns counting whole schema usage

def get_schema(attribute_name):
    levels = attribute_name.split('.')
    return levels[0]
    
count_schema = project_counts_df.reset_index()
count_schema['schema'] = count_schema['index'].apply(get_schema)
count_schema['Schema usage (no. project)'] = count_schema.groupby('schema')['True'].transform('max')
count_schema = count_schema.set_index(['index']).rename(index=str, columns={"True": "Attribute usage (no. projects)", "False": "Attribute NOT used (no. projects)", "schema" : "Schema Name"})
# count_schema.to_csv('project_counts.csv')
count_schema






,Attribute NOT used (no. projects),Attribute usage (no. projects),Schema Name,Schema usage (no. project)
index,,,,
sequencing_protocol.protocol_core.protocol_id,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.protocol_core.protocol_name,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.protocol_core.protocol_description,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.protocol_core.publication_doi,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.protocol_core.protocols_io_doi,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.protocol_core.document,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.instrument_manufacturer_model.text,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.instrument_manufacturer_model.ontology,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.instrument_manufacturer_model.ontology_label,0.0,15.0,sequencing_protocol,15.0


In [1]:
# Alternatively skip above steps and import from previous 'project_counts.csv'
import pandas as pd
count_schema = pd.read_csv('project_counts.csv', index_col = 0)
count_schema

,Attribute NOT used (no. projects),Attribute usage (no. projects),Schema Name,Schema usage (no. project)
index,,,,
sequencing_protocol.protocol_core.protocol_id,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.protocol_core.protocol_name,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.protocol_core.protocol_description,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.protocol_core.publication_doi,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.protocol_core.protocols_io_doi,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.protocol_core.document,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.instrument_manufacturer_model.text,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.instrument_manufacturer_model.ontology,0.0,15.0,sequencing_protocol,15.0
sequencing_protocol.instrument_manufacturer_model.ontology_label,0.0,15.0,sequencing_protocol,15.0


## 4.

Lets visualise the results with some interactive plots.

In [2]:
# format dataframe
schema_usage = (
                count_schema[['Schema Name', 'Schema usage (no. project)']]
                .drop_duplicates()
                .sort_values(by='Schema usage (no. project)', ascending = False)
                .set_index(['Schema Name'])
               )

# plotting

from bokeh.io import show, output_notebook, reset_output
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
import math


output_notebook()

# source = ColumnDataSource(schema_usage)
x = schema_usage.index.tolist()
y = schema_usage['Schema usage (no. project)'].tolist()


p = figure(plot_width=800, plot_height=600, x_range=x)

p.vbar(x=x, top=y, width=0.9, color='green')

p.title.text = 'Schema usage in datasets selected'
p.xaxis.axis_label = 'Schema'
p.yaxis.axis_label = 'No. of projects using schema'
p.xaxis.major_label_orientation = math.pi/2.6
p.yaxis.axis_label_text_font_size = "12pt"
p.xaxis.axis_label_text_font_size = "12pt"
p.xaxis.major_label_text_font_size = "11pt"

show(p)

Loading BokehJS ...

In [3]:

import bokeh.plotting as bp
from bokeh.io import show
from bokeh.models.ranges import FactorRange
from bokeh.models.glyphs import VBar
from ipywidgets import interact
output_notebook()

schema_names = list(set(count_schema['Schema Name'].tolist()))
f = schema_names[3] # first iteration


def get_data(f):
    
    data = (count_schema.loc[count_schema['Schema Name'] == f]
            .reset_index()
            .sort_values(by='Attribute usage (no. projects)', ascending = False))
    
    def shorter_name(attribute_name): # add column shorter attribute names missing top schema name
        levels = attribute_name.split('.')
        return '.'.join(levels[1:])
    data['x_label'] = data['index'].apply(shorter_name)
    return data


def plotter(data, f):

    source = ColumnDataSource(data)
    
    
    p = figure(plot_width=800,
               plot_height=600,
               x_range=FactorRange(factors=list(data['x_label'])),
               background_fill_color='#efefef')
    
    r = p.vbar(x="x_label", top="Attribute usage (no. projects)", source=source, bottom=0, width=0.5, fill_color="#b3de69")
    

    p.title.text = 'Schema usage in datasets selected'
    p.yaxis.axis_label = 'No. of projects using attribute'
    p.xaxis.major_label_orientation = math.pi/2.6
    p.xaxis.axis_label = f + ' attributes'
    p.yaxis.axis_label_text_font_size = "12pt"
    p.xaxis.axis_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "11pt"
    return (p)

def update(schema):
    data = get_data(schema)
    plotting = plotter(data, schema)
    p = plotting
    handle = show(p, notebook_handle=True)

print('Select schema from list.')
interact(update, schema=schema_names)

Loading BokehJS ...

Select schema from list.


interactive(children=(Dropdown(description='schema', options=('ipsc_induction_protocol', 'cell_line', 'referen…

<function __main__.update(schema)>